In [1]:
from calendar import monthrange
from datetime import datetime
import pandas as pd
from pandas.api.types import CategoricalDtype
from io import BytesIO
import os
import json

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
sys.path.append(os.path.abspath("../tools"))
from CloudIO.AzStorageClient import AzStorageClient
from edahelpers import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [2]:
root_dir =  os.path.abspath(os.getcwd()+"..\\..\\..\\..\\")
tmp_dir = root_dir + "\\.tmp"
data_dir = root_dir + "\\data\\"
cred_dir = root_dir + "\\.cred"
az_cred_file = cred_dir + '\\azblobcred.json'

site_metadata_filename = data_dir + "site-metadata.csv"

In [3]:
# "Golden" Sites
tier1_sites = ["US-MMS", "US-Vcp", "FR-Pue", "CH-Lae", "US-Var", "US-Ne2", "ES-LJu", "US-Ton"]
tier2_sites = ["US-UMB", "US-Me2", "FI-Hyy", "US-NR1", "IT-Lav", "US-Wkg", "US-ARM", "US-SRM"]

target_sites = tier1_sites + tier2_sites

# Get Gold Sample Site Data

In [4]:
# Load Site data
site_metadata_df = pd.read_csv(site_metadata_filename, usecols=['site_id','filename'])

# only focus on target sites
site_metadata_df= site_metadata_df.loc[site_metadata_df['site_id'].isin(target_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df

size:(16, 2)


,site_id,filename
67,FR-Pue,data_full_half_hourly_raw_v0_1_FR-Pue.csv
117,US-NR1,data_full_half_hourly_raw_v0_1_US-NR1.csv
119,US-Ne2,NaN
124,US-SRM,data_full_half_hourly_raw_v0_1_US-SRM.csv
127,US-Ton,data_full_half_hourly_raw_v0_1_US-Ton.csv
130,US-Var,data_full_half_hourly_raw_v0_1_US-Var.csv
144,US-Wkg,data_full_half_hourly_raw_v0_1_US-Wkg.csv
166,US-ARM,data_full_half_hourly_raw_v0_1_US-ARM.csv
181,US-MMS,NaN
182,US-Me2,data_full_half_hourly_raw_v0_1_US-Me2.csv


In [5]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']
qc_flag_dtype = CategoricalDtype([0, 1, 2, 3], ordered=True)
qc_flags_features = [s for s in all_features if "_QC" in s]

In [6]:
# Read files
data_df = None
for i, r in site_metadata_df[['site_id','filename']].iterrows():
    if not r.filename or type(r.filename) != type(""):
        print(f'\nERROR: {r.site_id} is mssing hourly data.')
        continue
    
    local_filename = tmp_dir + "\\" + r.filename
    site_df = pd.read_csv(local_filename)
    site_df['datetime'] = pd.to_datetime(site_df['datetime'])
    site_df['date'] = pd.to_datetime(site_df['date'])
    site_df[qc_flags_features] = site_df[qc_flags_features].astype(qc_flag_dtype)
    site_df['site_id'] = r.site_id
    
    print(f"{r.site_id}: {site_df.shape}")
    if type(data_df) == type(None):
        data_df = site_df
    else:
        data_df = pd.concat([data_df, site_df])

FR-Pue: (245760, 65)
US-NR1: (270768, 65)

ERROR: US-Ne2 is mssing hourly data.
US-SRM: (190752, 65)
US-Ton: (230928, 65)
US-Var: (245712, 65)
US-Wkg: (186768, 65)
US-ARM: (259104, 65)

ERROR: US-MMS is mssing hourly data.
US-Me2: (230688, 65)
US-UMB: (191904, 65)
US-Vcp: (174528, 65)
CH-Lae: (288384, 65)
ES-LJu: (239616, 65)
FI-Hyy: (407472, 65)
IT-Lav: (297840, 65)


In [7]:
245760+270768+190752+230928+245712+186768+259104+230688+191904+174528+288384+239616+407472+297840

3460224

In [8]:
data_df.shape

(3460224, 65)

In [10]:
data_df.site_id.unique()

array(['FR-Pue', 'US-NR1', 'US-SRM', 'US-Ton', 'US-Var', 'US-Wkg',
       'US-ARM', 'US-Me2', 'US-UMB', 'US-Vcp', 'CH-Lae', 'ES-LJu',
       'FI-Hyy', 'IT-Lav'], dtype=object)

## Upload Data to Azure Storage Blob as Parquet

In [11]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
parquet_file = BytesIO()
data_df.to_parquet(parquet_file, engine='pyarrow')
parquet_file.seek(0)

container = "gold-samples-data"
blob_name = "gold_samples_full_data.parquet"
azStorageClient = AzStorageClient(az_cred_file)
azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to gold-samples-data/gold_samples_full_data.parquet


# Download parquet data from Azure Storage Blob

In [12]:
del data_df

In [13]:
# Download the parquet file
# ref: https://stackoverflow.com/a/68940709
container = "gold-samples-data"
blob_name = "gold_samples_full_data.parquet"
azStorageClient = AzStorageClient(az_cred_file)
file_stream = azStorageClient.downloadBlob2Stream(container, blob_name)
data_df = pd.read_parquet(file_stream, engine='pyarrow')
print(f"data size: {data_df.shape}")
data_df.head()

data size: (3460224, 65)


,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,site_id
0,200007260000,200007260030,15.82,0,18.550,0.0,0.000,0,0.0,384.688,2,384.688,0.162,0,1.593,0.0,0.0,1.638,97.769,2.0,97.769,-36.32,0.419,NaN,NaN,4.80329,2.0,NaN,-19.6606,2.0,NaN,2.95832,2,2.95832,2,0.260122,0.000000,0.263564,0.000000,3.20784,2.26316,3.22063,1.78651,2000-07-26 00:00:00,2000,7,26,0,FR-Pue,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,FR-Pue
1,200007260030,200007260100,15.62,0,18.370,0.0,0.019,0,0.0,384.688,2,384.688,0.195,0,1.552,0.0,0.0,1.638,97.771,2.0,97.771,-32.59,0.519,NaN,NaN,5.04340,2.0,NaN,-19.6606,2.0,NaN,2.95832,2,2.95832,2,0.221846,0.000644,0.225435,0.000644,3.16912,2.25724,3.18245,1.78182,2000-07-26 00:30:00,2000,7,26,0,FR-Pue,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,FR-Pue
2,200007260100,200007260130,16.48,0,18.190,0.0,0.000,0,0.0,340.124,2,340.124,1.331,0,1.511,0.0,0.0,0.000,97.773,2.0,97.773,-38.81,-0.043,NaN,NaN,3.94853,1.0,NaN,-24.5284,2.0,NaN,3.03104,2,3.03104,2,0.315191,0.000000,0.318129,0.000000,3.33718,2.28257,3.34810,1.80182,2000-07-26 01:00:00,2000,7,26,1,FR-Pue,2000-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,FR-Pue
3,200007260130,200007260200,17.29,0,17.924,0.0,0.000,0,0.0,340.124,2,340.124,2.685,0,1.724,0.0,0.0,0.000,97.797,2.0,97.797,-45.83,-0.183,NaN,NaN,4.77351,1.0,NaN,-34.0815,1.0,NaN,4.31938,1,4.31938,1,-0.813255,0.000000,-0.810981,0.000000,3.49911,2.30605,3.50757,1.82036,2000-07-26 01:30:00,2000,7,26,1,FR-Pue,2000-07-26,4.31938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,FR-Pue
4,200007260200,200007260230,17.31,0,17.658,0.0,0.000,0,0.0,340.124,2,340.124,2.530,0,1.938,0.0,0.0,0.000,97.821,2.0,97.821,-45.37,-0.544,NaN,NaN,3.37369,1.0,NaN,-30.9598,1.0,NaN,4.34014,1,4.34014,1,-0.830025,0.000000,-0.827767,0.000000,3.50316,2.30663,3.51155,1.82082,2000-07-26 02:00:00,2000,7,26,2,FR-Pue,2000-07-26,4.34014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EBF,Temperate,FR-Pue


## View NA Data

In [17]:
total_record_count = data_df.shape[0]
na_df = pd.DataFrame(data_df.isna().sum())
na_df["percentage"] = (na_df / total_record_count)
na_df.rename(columns={0:"count"}, inplace=True)

In [18]:
na_df.loc[(na_df['count'] != 0)].sort_values("percentage", ascending=False)

,count,percentage
SW_DIF,3245894,0.938059
G_F_MDS,1049469,0.303295
G_F_MDS_QC,1049469,0.303295
H_CORR,1002477,0.289714
LE_CORR,1002477,0.289714
NETRAD,669801,0.193572
b6,500400,0.144615
LE_F_MDS_QC,470341,0.135928
H_F_MDS_QC,470341,0.135928
b5,403776,0.116691


## Get NA Percentage per Sites

In [139]:
sites = data_df['site_id'].unique()
features =  data_df.columns
plot_data = pd.DataFrame(columns=features[:-1])

for i, s in enumerate(data_df['site_id'].unique()):

    site_df = data_df[data_df['site_id'] == s]
    
    site_na_df = pd.DataFrame(site_df.isna().sum())
    site_na_df["percentage"] = round( 100*(site_na_df / site_df.shape[0]), 2)
    site_na_df.drop(["site_id"], inplace=True)
    #display(site_na_df.T)
    site_na_df.drop([0], axis=1, inplace=True)
    
    plot_data.loc[s] = site_na_df["percentage"]

plot_data

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,LW_IN_F_QC,LW_IN_ERA,VPD_F,VPD_F_QC,VPD_ERA,P_F,P_F_QC,P_ERA,PA_F,PA_F_QC,PA_ERA,NETRAD,PPFD_IN,G_F_MDS,G_F_MDS_QC,LE_F_MDS,LE_F_MDS_QC,LE_CORR,H_F_MDS,H_F_MDS_QC,H_CORR,NEE_VUT_REF,NEE_VUT_REF_QC,NEE_CUT_REF,NEE_CUT_REF_QC,GPP_NT_VUT_REF,GPP_DT_VUT_REF,GPP_NT_CUT_REF,GPP_DT_CUT_REF,RECO_NT_VUT_REF,RECO_DT_VUT_REF,RECO_NT_CUT_REF,RECO_DT_CUT_REF,datetime,year,month,day,hour,SITE_ID,date,NEE_VUT_REF_qa,SW_DIF,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
FR-Pue,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.76,9.05,41.61,41.61,0.0,0.00,20.41,0.0,0.00,20.41,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00,100.00,3.95,3.89,3.89,3.89,3.89,3.95,3.89,4.34,4.59,3.93,0.0,0.0
US-NR1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.66,0.55,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.13,100.00,21.41,21.34,21.34,21.34,21.34,21.41,21.38,21.57,28.59,21.34,0.0,0.0
US-SRM,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.39,0.40,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.87,100.00,0.13,0.08,0.08,0.08,0.08,0.13,0.08,0.08,0.18,0.13,0.0,0.0
US-Ton,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.43,0.10,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.00,100.00,0.96,0.96,0.96,0.96,0.96,0.96,0.96,1.00,2.41,0.96,0.0,0.0
US-Var,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.81,0.61,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.69,100.00,1.89,1.89,1.89,1.89,1.89,1.89,1.89,1.89,2.83,1.89,0.0,0.0
US-Wkg,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.47,0.53,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.86,100.00,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.26,0.31,0.26,0.0,0.0
US-ARM,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.85,8.38,4.42,4.42,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,6.0,0.00,6.0,0.00,6.0,0.00,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.39,82.42,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.30,2.61,1.19,0.0,0.0
US-Me2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.68,1.62,53.27,53.27,0.0,45.04,45.04,0.0,45.04,45.04,0.0,0.0,0.0,0.0,7.32,0.0,7.32,0.0,7.32,0.0,7.32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.72,100.00,7.24,7.24,7.24,7.24,7.24,7.24,7.24,7.51,10.57,7.26,0.0,0.0
US-UMB,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.90,15.36,100.00,100.00,0.0,100.00,100.00,0.0,100.00,100.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.50,100.00,28.46,28.44,28.44,28.44,28.44,28.46,28.44,29.39,31.64,28.44,0.0,0.0
US-Vcp,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,28.85,7.94,100.00,100.00,0.0,100.00,100.00,0.0,100.00,100.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.37,100.00,6.30,6.27,6.27,6.27,6.24,6.30,6.27,7.01,10.12,6.24,0.0,0.0


In [140]:
plot_df = plot_data.loc[:, (plot_data.sum(axis=0) != 0)]
plot_df.shape

(14, 38)

In [179]:
fig = px.imshow(plot_df,
                color_continuous_scale = 'amp',
                text_auto=False,
                title= "Proportions of Missing Data per Data Type and Site",
                labels=dict(x="Data Type", y="Site ID",
                            color="Missing<br>Data(%)"),
               )
fig.update(data=[{'hovertemplate': '%{y} (%{x})<br>Missing: %{z}%'}])
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title={'y':0.95,'x':0.025},
    coloraxis_colorbar=dict(
        title_font_size = 14,
        tickfont_size = 12,
        lenmode="pixels", len=300,
        thicknessmode="pixels", thickness=15
    )
)

fig.show()